# ライブドアニュースコーパスからデータ作成


In [1]:
tgz_fname = "ldcc-20140209.tar.gz"

# 分類用データへの加工

* tar.gzファイルから2つのジャンルを対象にする
  * text/GENRE/GENRE-#######.txt
    * 3行目がタイトルなのでこれを分類対象とする
      * 文章の前後についている【】を削除
* tsvファイルとして出力
  * フィールド構造
    1. (未使用)
    2. クラス (0/1)
    3. (未使用)
    4. テキスト

In [2]:
! mkdir swp

In [3]:
import tarfile
import csv
import re

target_genre = ["sports-watch", "peachy"] # 大きく異なるジャンル2つを選定

zero_fnames = []
one_fnames = []
tsv_fname = "swp/all.tsv"

brackets_tail = re.compile('【[^】]*】$')
brackets_head = re.compile('^【[^】]*】')

def remove_brackets(inp):
    output = re.sub(brackets_head, '',
                   re.sub(brackets_tail, '', inp))
    return output

def read_title(f):
    # 2行スキップ
    next(f)
    next(f)
    title = next(f) # 3行目を返す
    title = remove_brackets(title.decode('utf-8'))
    return title[:-1]

with tarfile.open(tgz_fname) as tf:
    # 対象ファイルの選定
    for ti in tf:
        # ライセンスファイルはスキップ
        if "LICENSE.txt" in ti.name:
            continue
        if target_genre[0] in ti.name and ti.name.endswith(".txt"):
            zero_fnames.append(ti.name)
            continue
        if target_genre[1] in ti.name and ti.name.endswith(".txt"):
            one_fnames.append(ti.name)
    with open(tsv_fname, "w") as wf:
        writer = csv.writer(wf, delimiter='\t')
        # ラベル 0
        for name in zero_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[0], 0, '', title]
            writer.writerow(row)
        # ラベル 1
        for name in one_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[1], 1, '', title]
            writer.writerow(row)


# データのシャッフルと分割

* train/dev/test : 8:1:1 で分割

In [4]:
! (cd swp && shuf all.tsv > rand-all.tsv)

In [5]:
import random

train_fname, dev_fname, test_fname = ["swp/train.tsv", "swp/dev.tsv", "swp/test.tsv"]

with open("swp/rand-all.tsv") as f, open(train_fname, "w") as tf, open(dev_fname, "w") as df, open(test_fname, "w") as ef:
    for line in f:
        v = random.randint(0, 9)
        if v == 8:
            df.write(line)
        elif v == 9:
            ef.write(line)
        else:
            tf.write(line)

BERTモデルのセットアップは [0-setup](0-setup.ipynb)を参照